# Code to run MD on polymers using Langevin dynamics

In [ ]:
from ase.io import read, write
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin


import random
import os
import time
import numpy as np
import pylab as pl
from IPython import display

from mace.calculators import MACECalculator
# Path for mace_calc for running in lorca not my PC, hence path for this kernal is not right
mace_calc = MACECalculator(model_paths=['/home/jaivin/src/mace/mace/calculators/foundations_models/2023-12-03-mace-mp.model'], device='cpu', default_dtype="float32")

from ase.io.proteindatabank import read_proteindatabank as read_pdb
from ase.io.proteindatabank import write_proteindatabank as write_pdb

def simpleMD(init_conf, temp, calc, fname, s, T):
    init_conf.set_calculator(mace_calc)

    # Initialize the temperature
    random.seed(701) # just making sure the MD failure is reproducible
    MaxwellBoltzmannDistribution(init_conf, temperature_K=temp) # initialize temperature at 300 K

    dyn = Langevin(init_conf, 1.0*units.fs, temperature_K=temp, friction=0.1) #drive system to desired temperature

    #remove previously stored trajectory with the same name
    os.system('rm -rfv '+fname)

    def write_frame():
        time_fs = []
        energies = []

        dyn.atoms.write(fname, append=True)
        time_fs.append(dyn.get_time()/units.fs)
        energies.append(dyn.atoms.get_potential_energy()/len(dyn.atoms))
        time.sleep(0.01)

    dyn.attach(write_frame, interval=s)

    
    t0 = time.time()
    
    dyn.run(T)
    
    t1 = time.time()
    print("MD finished in {0:.2f} minutes!".format((t1-t0)/60))


# Run MD. CHANGE FILENAME BEFORE RUNNING
polymer = read_pdb('/home/jaivin/awe_projects/polymer_optimisations/pdms1_opt.pdb')
simpleMD(polymer, temp=300, calc=mace_calc, fname='polymer_md_29-07.xyz', s=10, T=2000)

# Code to run Berendsen NVT MD

In [ ]:
from ase.io import read, write
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nvtberendsen import NVTBerendsen

import random
import os
import time
from IPython import display

from mace.calculators import MACECalculator

# Path for mace_calc for running in lorca not my PC, hence path for this kernel is not right
mace_calc = MACECalculator(model_paths=['/home/jaivin/src/mace/mace/calculators/foundations_models/2023-12-03-mace-mp.model'], device='cpu', default_dtype="float32")

def simpleMD(init_conf, temp, calc, fname, s, T):
    init_conf.set_calculator(calc)

    # Initialize the temperature
    random.seed(701)  # Just making sure the MD failure is reproducible
    MaxwellBoltzmannDistribution(init_conf, temperature_K=temp)  # Initialize temperature at 300 K

    dyn = NVTBerendsen(init_conf, 1.0 * units.fs, temperature_K=temp, taut= 5*1000*units.fs) 

    # Remove previously stored trajectory with the same name
    if os.path.exists(fname):
        os.remove(fname)

    def write_frame():
        dyn.atoms.write(fname, append=True)
        time.sleep(0.01)

    dyn.attach(write_frame, interval=s)

    t0 = time.time()
    
    dyn.run(T)
    
    t1 = time.time()
    print("MD finished in {0:.2f} minutes!".format((t1 - t0) / 60))

# Read the final state from Langevin MD
polymer = read('pdms2_md_29-07.xyz', index=-1)

# Run MD, using final state from Langevin MD. CHANGE FILENAME BEFORE RUNNING
simpleMD(polymer, temp=300, calc=mace_calc, fname='pdms2_berendsenNVT_md_05-08.xyz', s=10, T=2000)

# Code to run Berendsen NPT MD at atmospheric pressure

In [ ]:
from ase.io import read, write
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen

import random
import os
import time
from IPython import display

from mace.calculators import MACECalculator

# Path for mace_calc for running in lorca not my PC, hence path for this kernel is not right
mace_calc = MACECalculator(model_paths=['/home/jaivin/src/mace/mace/calculators/foundations_models/2023-12-03-mace-mp.model'], device='cpu', default_dtype="float32")

def simpleMD(init_conf, temp, calc, fname, s, T):
    init_conf.set_calculator(calc)

    # Initialize the temperature
    random.seed(701)  # Just making sure the MD failure is reproducible
    MaxwellBoltzmannDistribution(init_conf, temperature_K=temp)  # Initialize temperature at 300 K

    dyn = NPTBerendsen(init_conf, 1.0 * units.fs, temperature_K=temp, taut= 5*1000*units.fs, pressure_au = 1.01325, compressibility_au = 1.3532e-4 / units.bar) 

    # Remove previously stored trajectory with the same name
    if os.path.exists(fname):
        os.remove(fname)

    def write_frame():
        dyn.atoms.write(fname, append=True)
        time.sleep(0.01)

    dyn.attach(write_frame, interval=s)

    t0 = time.time()
    dyn.run(T)
    t1 = time.time()
    print("MD finished in {0:.2f} minutes!".format((t1 - t0) / 60))

# Read the final state from Langevin MD
polymer = read('pdms2_berendsenNVT_md_05-08.xyz', index=-1)

# Run MD, using final state from Langevin MD. CHANGE FILENAME BEFORE RUNNING
simpleMD(polymer, temp=300, calc=mace_calc, fname='pdms2_berendsenNPT_md_06-08.xyz', s=10, T=2000)